https://gis.stackexchange.com/questions/461143/convert-wrf-netcdf-to-tiff

In [141]:
import os
import netCDF4 as nc
import numpy as np
import rasterio
from rasterio.transform import from_origin
import wrf
import xarray as xr
import numpy as np

import cartopy.crs as crs
import cartopy.crs as ccrs

In [142]:
folder = 'wrfout'
for filename in os.listdir(folder):
    # Check if the filename contains a colon
    if '%' in filename or ':' in filename:
        new_filename = filename.replace("%", "-")
        new_filename = filename.replace(":", "-")
        old_path = os.path.join(folder, filename)
        new_path = os.path.join(folder, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")

In [144]:
root = 'wrfout'
data = nc.Dataset(root + '/wrfout_d03_2025-03-04_08-3A00-3A00')

In [145]:
t2 =  wrf.getvar(data, "T2", timeidx=0)
t2

<xarray.DataArray 'T2' (south_north: 100, west_east: 100)> Size: 40kB
array([[279.6635 , 279.5325 , 279.35266, ..., 288.20364, 288.25037,
        288.29462],
       [279.42566, 279.31183, 279.15912, ..., 288.17953, 288.22162,
        288.26483],
       [279.16684, 279.06964, 278.9292 , ..., 288.13995, 288.19287,
        288.23425],
       ...,
       [286.93857, 287.01212, 287.03683, ..., 290.30173, 290.30606,
        290.30823],
       [287.17984, 287.20514, 287.37054, ..., 290.30222, 290.30698,
        290.30927],
       [287.31924, 287.42252, 287.52487, ..., 290.3012 , 290.30576,
        290.3093 ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT     (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

In [180]:
u10 = wrf.getvar(data, 'U10', timeidx=0)  
v10 = wrf.getvar(data, 'V10', timeidx=0)

nx = data.dimensions['west_east'].size
ny = data.dimensions['south_north'].size
dt, dx, dy = data.DT, data.DX, data.DY
cen_lat, cen_lon = data.CEN_LAT, data.CEN_LON  # center of the domain
truelat1, truelat2 = data.TRUELAT1, data.TRUELAT2  # true latitudes
pole_lat, pole_lon = data.POLE_LAT, data.POLE_LON  # coordinates of the pole    

AttributeError: 'list' object has no attribute 'dim'

In [ ]:
cone = 1.0
uv = wrf.uvmet(u10, v10, u10.XLONG, u10.XLAT, cone, cen_lon, meta=True, units=' m s-1')
uv.shape

AttributeError: module 'wrf' has no attribute 'wspd_wdir'

In [197]:
uv[1][0,0]

<xarray.DataArray 'uvmet' ()> Size: 4B
array(-1.890662, dtype=float32)
Coordinates:
    u_v      <U1 4B 'v'
Attributes:
    units:         m s-1
    description:  earth rotated u,v

In [148]:
xmin = float(np.min(t2.XLONG).values)
xmax = float(np.max(t2.XLONG).values)
ymin = float(np.min(t2.XLAT).values)
ymax = float(np.max(t2.XLAT).values)


print( xmin, xmax, ymin, ymax)

-15.61920166015625 -15.390350341796875 27.948341369628906 28.15030288696289


In [181]:
#Extracting coordinates
latitudes = data.variables["XLAT"][0,:,:]
longitudes = data.variables["XLONG"][0,:,:]

# take a single time step
time0_T2 = data.variables["T2"][0,:,:]

# take a signgle wind step
time0_U10 = data.variables["U10"][0,:,:]
time0_V10 = data.variables["V10"][0,:,:]

# defining lists for the output dataset
lat, lon, temp, u10, v10, nc = [], [], [], [], [], []

In [168]:
print(time0_T2.shape), print(time0_U10.shape), print(time0_V10.shape)   

(100, 100)
(100, 100)
(100, 100)


(None, None, None)

In [150]:
data.variables["XLAT"]

<class 'netCDF4.Variable'>
float32 XLAT(Time, south_north, west_east)
    FieldType: 104
    MemoryOrder: XY 
    description: LATITUDE, SOUTH IS NEGATIVE
    units: degree_north
    stagger: 
    coordinates: XLONG XLAT
unlimited dimensions: Time
current shape = (1, 100, 100)
filling on, default _FillValue of 9.969209968386869e+36 used

In [151]:
print(latitudes.shape)
print(longitudes.shape)

(100, 100)
(100, 100)


In [152]:
latitudes

masked_array(
  data=[[27.950848, 27.950829, 27.95081 , ..., 27.948391, 27.948376,
         27.948341],
        [27.952866, 27.952835, 27.952816, ..., 27.950397, 27.950386,
         27.950352],
        [27.95488 , 27.954853, 27.954838, ..., 27.952427, 27.952385,
         27.952366],
        ...,
        [28.146267, 28.14624 , 28.146221, ..., 28.143803, 28.143776,
         28.143753],
        [28.148277, 28.148254, 28.148243, ..., 28.145817, 28.14579 ,
         28.145771],
        [28.150303, 28.150265, 28.150238, ..., 28.147839, 28.147804,
         28.147778]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)

In [182]:
# counter variable for cell indexing
count = 1
for i in range(latitudes.shape[0]):
    for j in range(latitudes.shape[1]):        
        latitude = latitudes[i, j]
        longitude = longitudes[i, j]
        if ymin <= latitude <= ymax and xmin <= longitude <= xmax:               
            lat.append(latitude)
            lon.append(longitude)
            # convert temperature from Kelvin to Celsius
            temp.append(time0_T2[i, j])
            u10.append(time0_U10[i, j])
            v10.append(time0_V10[i, j])
            nc.append(count)
            count = count + 1

In [183]:
# converting to numpy column array
A = np.transpose(np.array([nc,temp,u10, v10, lon, lat]))
A

array([[ 1.00000000e+00,  2.79663513e+02, -6.22136414e-01,
        -4.54551268e+00, -1.56192017e+01,  2.79508476e+01],
       [ 2.00000000e+00,  2.79532501e+02, -9.00985181e-01,
        -4.37349129e+00, -1.56169128e+01,  2.79508286e+01],
       [ 3.00000000e+00,  2.79352661e+02, -1.14835501e+00,
        -4.34228897e+00, -1.56146240e+01,  2.79508095e+01],
       ...,
       [ 9.99800000e+03,  2.90301208e+02,  1.54791331e+00,
        -5.40307760e+00, -1.53949280e+01,  2.81478386e+01],
       [ 9.99900000e+03,  2.90305756e+02,  1.53800833e+00,
        -5.44134903e+00, -1.53926392e+01,  2.81478043e+01],
       [ 1.00000000e+04,  2.90309296e+02,  1.53450012e+00,
        -5.46945477e+00, -1.53903503e+01,  2.81477776e+01]])

In [ ]:
# writing data to csv
np.savetxt('time0_T2.csv',A,delimiter=",",header="cell_number,T2_degC,U10, V10, LON,LAT")

In [156]:
# x and y lenghts
nx = np.unique(np.round(A[:,2],2)).shape[0]
ny = np.unique(np.round(A[:,3],3)).shape[0]

print(nx, ny)

24 203


In [157]:
# x and y resolutions
deltax = (np.max(A[:,2]) - np.min(A[:,2]))/(nx-1)
deltay = (np.max(A[:,3]) - np.min(A[:,3]))/(ny-1)

print(deltax, deltay)

0.009950057319972826 0.0009998094917523979


In [158]:
# up left corner
minx = np.min(A[:,2])
maxy = np.max(A[:,3])
print(minx, maxy)

-15.61920166015625 28.15030288696289


In [159]:
# set the transformation for rasterio. Note that the shift operated is in order to
# make the lat,lon points to be the center of each grid cell
transform = from_origin(minx-0.5*deltax,maxy+0.5*deltay, deltax, deltay)

In [160]:
A[:,1].shape


(10000,)

In [161]:
# write it to tiff format
with rasterio.open('time0_T2.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=A[:,1].dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(A[:,1].reshape(1, ny,nx))

CPLE_AppDefinedError: Deleting time0_T2.tiff failed: Permission denied